In [ ]:
!unzip /kaggle/input/dogs-vs-cats/train.zip

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
input_path = []
labels = []
for image_path in os.listdir('train'):
    if 'cat' in image_path:
        labels.append(0)
    else:
        labels.append(1)
    input_path.append('train/'+image_path)

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['labels'] = labels
df = df.sample(frac = 1)

In [ ]:
df['labels'] = df['labels'].astype('str')

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state = 42)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator = ImageDataGenerator(
    rescale = 1./255,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
val_image_generator = ImageDataGenerator(
    rescale = 1./255
)

train = train_image_generator.flow_from_dataframe(train, x_col = 'images', y_col = 'labels', batch_size = 256, target_size = (224,224), class_mode = 'binary')
test = val_image_generator.flow_from_dataframe(test, x_col = 'images', y_col = 'labels', batch_size = 256, target_size = (224,224), class_mode = 'binary')


In [ ]:
print(len(train[0])) # train[i] => batch thứ i, mỗi batch có 2 thuộc tính là image và label, train[0][1] => label
print(train[0][1])

In [ ]:
from keras import layers
from keras import Sequential

input = keras.Input(shape = (224,224,3))
def res_block(x, filters, strides):
    if strides != 1:
        short_cut = layers.Conv2D(filters, (1,1), strides = strides, padding = 'same')(x)
    else:
        short_cut = x
    x = layers.Conv2D(filters, (3,3), strides = strides, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(filters, (3,3), strides = 1, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    out = layers.Add()([short_cut, x])
    return out
x = layers.Conv2D(64, (7,7), strides = 2)(input)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(strides = 2)(x)
for i in range(3):
    x = res_block(x, 64, 1)
x = res_block(x, 128, 2)
for i in range(3):
    x = res_block(x, 128, 1)
x = res_block(x, 256, 2)
for i in range(5):
    x = res_block(x, 256, 1)
x = res_block(x, 512, 2)
for i in range(2):
    x = res_block(x, 512, 1)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation = 'relu')(x)
output = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs = input, outputs = output)
model.summary()

In [ ]:
from keras import layers
from keras import Sequential

input = keras.Input(shape = (224,224,3))

def res_block1(x):
    init_x = x
    x = layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
    x = layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)
    return x + init_x
def res_block2(x):
    init_x = x
    x = layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
    x = layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
    return x + init_x
def res_block3(x):
    init_x = x
    x = layers.Conv2D(256, (3,3), padding = 'same', activation = 'relu')(x)
    x = layers.Conv2D(256, (3,3), padding = 'same', activation = 'relu')(x)
    return x + init_x
def res_block4(x):
    init_x = x
    x = layers.Conv2D(512, (3,3), padding = 'same', activation = 'relu')(x)
    x = layers.Conv2D(512, (3,3), padding = 'same', activation = 'relu')(x)
    return x + init_x
x = layers.Conv2D(64, (7,7), strides = 2, activation = 'relu')(input)
x = layers.MaxPooling2D((2,2), strides = 2)(x)
for i in range(3):
    x = res_block1(x)
short_cut = layers.Conv2D(128, (1,1), strides = 2, padding = 'same')(x)
x = layers.Conv2D(128, (3,3), strides = 2, padding = 'same', activation = 'relu')(x)
x = layers.Conv2D(128, (3,3), padding = 'same', activation = 'relu')(x)
x = x + short_cut
for i in range(3):
    x = res_block2(x)
short_cut = layers.Conv2D(256, (1,1), strides = 2, padding = 'same')(x)
x = layers.Conv2D(256, (3,3), strides = 2, padding = 'same', activation = 'relu')(x)
x = layers.Conv2D(256, (3,3), padding = 'same', activation = 'relu')(x)
x = x + short_cut
for i in range(5):
    x = res_block3(x)
short_cut = layers.Conv2D(512, (1,1), strides = 2, padding = 'same')(x)
x = layers.Conv2D(512, (3,3), strides = 2, padding = 'same', activation = 'relu')(x)
x = layers.Conv2D(512, (3,3), padding = 'same', activation = 'relu')(x)
x = x + short_cut
for i in range(2):
    x = res_block4(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation = 'relu')(x)
output = layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs = input, outputs = output)
model.summary()

In [ ]:
from keras import layers
from keras import Sequential

In [ ]:
# Residual block
def residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    if conv_shortcut:
        shortcut = layers.Conv2D(filters=filters,kernel_size=(1, 1),strides=stride,padding='same')(x)
    else:
        shortcut = layers.Conv2D(filters=filters,kernel_size=(3, 3),strides=stride,padding='same')(x)
    out = layers.BatchNormalization()(shortcut)
    out = layers.ReLU()(out)
    out = layers.Conv2D(filters=filters,kernel_size=(3, 3),strides=1,padding='same')(out)
    out = layers.BatchNormalization()(out)
    out = layers.ReLU()(out)
    out = layers.Add()([shortcut, out])
    return out

# Model implementation
input = keras.Input(shape=(224, 224, 3))
initializer = tf.keras.initializers.HeNormal(seed=0)
x = layers.Conv2D(filters=64, kernel_size=(7, 7),strides=2, kernel_initializer=initializer,
                  use_bias=True, bias_initializer=keras.initializers.Zeros(), padding='same')(input)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
# Residual block implementation

# 1st residual block (64 channels)
for i in range(3):
    x = residual_block(x, filters=64, kernel_size=3, stride=1, conv_shortcut=False)

# 2nd residual block (128 channels)
x = residual_block(x, filters=128, kernel_size=3, stride=2, conv_shortcut=True)
for i in range(3):
    x = residual_block(x, filters=128, kernel_size=3, stride=1, conv_shortcut=False)
    
# 3rd residual block (256 channels)
x = residual_block(x, filters=256, kernel_size=3, stride=2, conv_shortcut=True)
for i in range(5):
    x = residual_block(x, filters=256, kernel_size=3, stride=1, conv_shortcut=False)
    
# 4th residual block (512 channels)
x = residual_block(x, filters=512, kernel_size=3, stride=2, conv_shortcut=True)
for i in range(2):
    x = residual_block(x, filters=512, kernel_size=3, stride=1, conv_shortcut=False)

# Fully connected layers
x = layers.GlobalAveragePooling2D()(x)
output = layers.Dense(1, activation='softmax', kernel_initializer=initializer)(x)

model = keras.Model(inputs=input, outputs=output, name="ResNet34_model")
model.summary()

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.00001)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(train, epochs = 20, validation_data = test)